# Bereinigung der einzelnen Datensätze

## Importieren der Datensätze und Pythonmodule

In [1]:
import pandas as pd
import numpy as np

df_besucher = pd.read_csv('data/besucher.csv', decimal = ',', sep = ";")   
df_kunden = pd.read_csv('data/kunden.csv', decimal = '.', sep = ";")
df_geo = pd.read_csv('data/geo.txt', decimal = '.', delimiter = "\t")

### Bereinigung von df_geo

In [2]:
df_geo['Niederlassung'].value_counts()

Niederlassung
Baden-Württemberg        382
Bayern                   338
Nordrhein-Westfalen      322
Berlin                   230
Niedersachsen            169
Hessen                   165
Sachsen                  162
Thüringen                 89
Hamburg                   67
Brandenburg               60
NRW                        3
Düsseldorf                 2
Berlin-Charlottenburg      2
Berlin-Mitte               2
BERLIN                     2
Name: count, dtype: int64

In [3]:
df_geo['Niederlassung']=df_geo['Niederlassung'].replace({'NRW':'Nordrhein-Westfalen'})
df_geo['Niederlassung']=df_geo['Niederlassung'].replace({'Düsseldorf':'Nordrhein-Westfalen'})
df_geo['Niederlassung']=df_geo['Niederlassung'].replace({'Berlin-Charlottenburg':'Berlin'})
df_geo['Niederlassung']=df_geo['Niederlassung'].replace({'Berlin-Mitte':'Berlin'})
df_geo['Niederlassung']=df_geo['Niederlassung'].replace({'BERLIN':'Berlin'})
df_geo['Niederlassung'].value_counts()

Niederlassung
Baden-Württemberg      382
Bayern                 338
Nordrhein-Westfalen    327
Berlin                 236
Niedersachsen          169
Hessen                 165
Sachsen                162
Thüringen               89
Hamburg                 67
Brandenburg             60
Name: count, dtype: int64

### Betrachtung Fehler bei Kunden und Besucher

In [56]:
df_kunden.isnull().sum()

Alter         0
Einkommen     0
Preis         0
Geschlecht    5
Zeit          0
KundeNr       0
dtype: int64

In [5]:
df_besucher.isnull().sum()
# df_besucher.dtypes

Alter         0
Einkommen     0
Geschlecht    0
Zeit          0
KundeNr       0
dtype: int64

### Zusammenführung von besucher.csv und kunden.csv

In [57]:
df_alle_ohneGEO = pd.concat([df_besucher,df_kunden], ignore_index=True,)
df_alle_ohneGEO_sortiert = df_alle_ohneGEO.sort_values(by=['KundeNr'],ascending=True).round(decimals=2)

df_alle_ohneGEO_sortiert

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis
1367,68,89437.00,1.0,62.0,K0001,114481.90
1783,58,65271.00,1.0,44.0,K0002,106258.99
1116,44,48195.00,1.0,29.0,K0003,85705.57
1065,37,48957.00,0.0,36.0,K0004,76412.36
1882,50,56621.00,0.0,40.0,K0005,92898.67
...,...,...,...,...,...,...
18,48,59967.31,1.0,21.0,K1991,NaN
799,67,81521.76,1.0,21.0,K1992,NaN
592,40,41616.57,0.0,48.0,K1993,NaN
72,49,76602.79,1.0,19.0,K1994,NaN


### Zusammenführung von geo.txt und df_alle_ohneGEO

In [7]:
df_final= pd.merge(df_alle_ohneGEO_sortiert,df_geo,on='KundeNr', how='outer').set_index('KundeNr')
df_final

,Alter,Einkommen,Geschlecht,Zeit,Preis,Niederlassung
KundeNr,,,,,,
K0001,68,89437.00,1.0,62.0,114481.90,Sachsen
K0002,58,65271.00,1.0,44.0,106258.99,Nordrhein-Westfalen
K0003,44,48195.00,1.0,29.0,85705.57,Hessen
K0004,37,48957.00,0.0,36.0,76412.36,Bayern
K0005,50,56621.00,0.0,40.0,92898.67,Bayern
...,...,...,...,...,...,...
K1991,48,59967.31,1.0,21.0,NaN,Hessen
K1992,67,81521.76,1.0,21.0,NaN,Baden-Württemberg
K1993,40,41616.57,0.0,48.0,NaN,Nordrhein-Westfalen


### F3

In [8]:
Gesamtumsatz = df_final['Preis'].sum()
print(Gesamtumsatz)

98228619.38


### F6

In [30]:
# Finde Index des Kunden mit maximalen Preis
index_MaxPreis = df_final['Preis'].idxmax()
print(df_final.loc[index_MaxPreis][5])

Niedersachsen


### F9

In [54]:
DurchschnittseinkommenKunden = df_final.loc[df_final['Preis'] != 'NaN','Einkommen'].mean()
print(df_final.loc[df_final['Preis'] >= 0, 'Einkommen'])
print('Das Durchschnittseinkommen unserer Kunden ist: '+ str(DurchschnittseinkommenKunden))

DurchschnittseinkommenBesucher = df_final.loc[df_final['Preis'] == 'NaN','Einkommen'].mean()
print('Das Durchschnittseinkommen unserer Besucher ist: '+ str(DurchschnittseinkommenBesucher))

KundeNr
K0001    89437.0
K0002    65271.0
K0003    48195.0
K0004    48957.0
K0005    56621.0
          ...   
K1100    82624.0
K1101    37905.0
K1102    40159.0
K1103    45545.0
K1104    80414.0
Name: Einkommen, Length: 1104, dtype: float64
Das Durchschnittseinkommen unserer Kunden ist: 560239.1126065163
Das Durchschnittseinkommen unserer Besucher ist: nan


In [41]:
df_final.dtypes

Alter              int64
Einkommen        float64
Geschlecht       float64
Zeit             float64
Preis            float64
Niederlassung     object
dtype: object